Создание сводных таблиц



In [14]:
import pandas as pd
import os
from docxtpl import DocxTemplate
import csv
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import openpyxl
import time
from openpyxl.chart.label import DataLabelList
from openpyxl.chart import BarChart, Reference, PieChart, PieChart3D, Series
# Показываем все колонки
pd.set_option('display.max_columns', None)

In [15]:
# Функции для создания сводной таблицы
def counting_total_student(dpo_df, po_df):
    """
    Функция для подсчета общего количества студентов обучающихся в цопп, и количества обучающихся по ДПО И ПО
    :param dpo_df: датафрейм ДПО
    :param po_df: датафрейм ПО
    :return: кортеж вида: общее количество обучающихся,количество обучающихся ДПО,количество обучающихся ПО
    """
    # количество по типам
    total_dpo = dpo_df.shape[0]
    total_po = po_df.shape[0]
    # общее количество
    total = total_dpo + total_po

    return total, total_dpo, total_po


def counting_type_of_training(dpo, po):
    """
    Функция для создания сводной таблицы по категориям направление подготовки, название программы,количество обучающихся
    :param dpo: датафрейм ДПО
    :param po: датафрейм ПО
    :return: датафрейм сводной таблицы
    """
    # Создаем сводные таблицы проверяее перед этим не пустые ли таблицы
    if dpo.shape[0] > 0:
        dpo_svod_category_and_name = pd.pivot_table(dpo, index=[
            'Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка',
            'Наименование_дополнительной_профессиональной_программы'],
                                                    values=['ФИО_именительный'],
                                                    aggfunc='count')
        dpo_svod_category_and_name = dpo_svod_category_and_name.reset_index()
    else:
        dpo_svod_category_and_name = pd.DataFrame(
            columns=['Направление подготовки', 'Название программы', 'Количество обученных'])

    if po.shape[0] > 0:
        po_svod_category_and_name = pd.pivot_table(po,
                                                   index=['Программа_профессионального_обучения_направление_подготовки',
                                                          'Наименование_программы_профессионального_обучения'],
                                                   values=['ФИО_именительный'],
                                                   aggfunc='count')
        po_svod_category_and_name = po_svod_category_and_name.reset_index()

    else:
        po_svod_category_and_name = pd.DataFrame(
            columns=['Направление подготовки', 'Название программы', 'Количество обученных'])
    # Изменяем названия колонок, чтобы без проблем соединить 2 датафрейма
    dpo_svod_category_and_name.columns = ['Направление подготовки', 'Название программы', 'Количество обученных']
    po_svod_category_and_name.columns = ['Направление подготовки', 'Название программы', 'Количество обученных']
    # Создаем единую сводную таблицу
    general_svod_category_and_name = pd.concat([dpo_svod_category_and_name, po_svod_category_and_name],
                                               ignore_index=True)
    return general_svod_category_and_name


def counting_total_sex(dpo, po):
    """
    Функция для подсчета количества мужчин и женщин
    :param dpo: датафрейм ДПО
    :param po: датафрейм ПО
    :return: датафрейм сводной таблицы
    """
    # Создаем сводные таблицы Проверяем на пустой лист ДПО или ПО
    if dpo.shape[0] > 0:

        dpo_total_sex = pd.pivot_table(dpo, index=['Пол_получателя'],
                                       values=['ФИО_именительный'],
                                       aggfunc='count')
        dpo_total_sex = dpo_total_sex.reset_index()
    else:
        dpo_total_sex = pd.DataFrame(columns=['Пол', 'Количество'])

    if po.shape[0] > 0:
        po_total_sex = pd.pivot_table(po, index=['Пол_получателя'],
                                      values=['ФИО_именительный'],
                                      aggfunc='count')
        po_total_sex = po_total_sex.reset_index()
    else:
        po_total_sex = pd.DataFrame(columns=['Пол', 'Количество'])
    # Переименовываем колонки
    dpo_total_sex.columns = ['Пол', 'Количество']
    po_total_sex.columns = ['Пол', 'Количество']

    # Соединяем в единую таблицу
    general_total_sex = pd.concat([dpo_total_sex, po_total_sex], ignore_index=True)
    # Группируем по полю Пол чтобы суммировать значения
    sum_general_total_sex = general_total_sex.groupby(['Пол']).sum().reset_index()
    return sum_general_total_sex


def counting_age_distribution_dpo(dpo):
    """
    Функция для подсчета количества обучающихся по возрастным категориям
    :param dpo: датафрейм ДПО
    :return: датафрейм сводной таблицы
    """
    # Создаем сводные таблицы
    if dpo.shape[0] > 0:
        dpo_age_distribution = pd.pivot_table(dpo, index=['Возрастная_категория_1ПК'],
                                              values=['ФИО_именительный'],
                                              aggfunc='count')
        dpo_age_distribution = dpo_age_distribution.reset_index()
    else:
        dpo_age_distribution = pd.DataFrame(columns=['Возрастная_категория_1ПК', 'Количество'])

    return dpo_age_distribution


def counting_age_distribution_po(po):
    """
    Функция для подсчета количества обучающихся по возрастным категориям
    :param dpo: датафрейм ПО
    :return: датафрейм сводной таблицы
    """
    if po.shape[0] > 0:
        po_age_distribution = pd.pivot_table(po, index=['Возрастная_категория_1ПО'],
                                             values=['ФИО_именительный'],
                                             aggfunc='count')
        po_age_distribution = po_age_distribution.reset_index()
    else:
        po_age_distribution = pd.DataFrame(columns=['Возрастная_категория_1ПО', 'Количество'])
    return po_age_distribution

In [16]:
name_file_data_report = 'Общая таблица слушателей ЦОПП от 21_06_22.xlsx'
path_to_end_folder_report = 'data'

In [17]:
dpo_df = pd.read_excel(name_file_data_report, sheet_name='ДПО',
                       dtype={'Гражданство_получателя_код_страны_по_ОКСМ': str})
po_df = pd.read_excel(name_file_data_report, sheet_name='ПО',
                      dtype={'Гражданство_получателя_код_страны_по_ОКСМ': str})
"""
Проверяем заполнена ли колонка Возрастная категория.Если заполнена, то значит таблица прошла через процедуру create_general_table
Но нужно обработать случай когда нужно сделать отчет по одной таблице
"""
if 'Текущий_возраст' not in dpo_df.columns or 'Текущий_возраст' not in po_df.columns:
    dpo_df['Текущий_возраст'] = dpo_df['Дата_рождения_получателя'].apply(calculate_age)
    dpo_df['Возрастная_категория_1ПК'] = pd.cut(dpo_df['Текущий_возраст'],
                                                [0, 24, 29, 34, 39, 44, 49, 54, 59, 64, 101, 10000],
                                                labels=['моложе 25 лет', '25-29', '30-34', '35-39',
                                                        '40-44', '45-49', '50-54', '55-59', '60-64',
                                                        '65 и более', 'Возраст  больше 101'])
    #
    po_df['Текущий_возраст'] = po_df['Дата_рождения_получателя'].apply(calculate_age)
    po_df['Возрастная_категория_1ПО'] = pd.cut(po_df['Текущий_возраст'],
                                               [0, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                                                28,
                                                29, 34, 39, 44, 49, 54, 59, 64, 101],
                                               labels=['В возрасте моложе 14 лет', '14 лет', '15 лет', '16 лет',
                                                       '17 лет', '18 лет', '19 лет', '20 лет', '21 год',
                                                       '22 года',
                                                       '23 года', '24 года', '25 лет',
                                                       '26 лет', '27 лет', '28 лет', '29 лет', '30-34 лет',
                                                       '35-39 лет', '40-44 лет', '45-49 лет', '50-54 лет',
                                                       '55-59 лет',
                                                       '60-64 лет',
                                                       '65 лет и старше'])
    # Приводим Возрастную категорию к текстовому типу, иначе при fillna возникает ошибка, он не может заполнить категориальные данные
    dpo_df['Возрастная_категория_1ПК'] = dpo_df['Возрастная_категория_1ПК'].astype(str)
    po_df['Возрастная_категория_1ПО'] = po_df['Возрастная_категория_1ПО'].astype(str)

# Заполняем пустые поля для удобства группировки
dpo_df = dpo_df.fillna('Не заполнено!!!')
po_df = po_df.fillna('Не заполнено!!!')

# Получение общего количества прошедших обучение,количества прошедших по ДПО,по ПО
total_students, total_students_dpo, total_students_po = counting_total_student(dpo_df, po_df)

# Количество обучившихся ДПО и ПО

# Получение количества обучившихся по видам
df_counting_type_and_name_trainning = counting_type_of_training(dpo_df, po_df)

# Создаем новый excel файл
wb = openpyxl.Workbook()

# Получаем активный лист
sheet = wb.active
sheet.title = 'Сводные данные'

# Начинаем заполнение листа
# Заполняем количество обучившихся, общее и по типам
sheet['A1'] = 'Наименование показателя'
sheet['A2'] = 'Количество прошедших обучение ДПО'
sheet['A3'] = 'Количество прошедших обучение ПО'
sheet['A4'] = 'Общее количество прошедших обучение в ЦОПП'

sheet['B1'] = 'Количество обучившихся'
sheet['B2'] = total_students_dpo
sheet['B3'] = total_students_po
sheet['B4'] = total_students

# Добавляем круговую диаграмму
pie_main = PieChart()
labels = Reference(sheet, min_col=1, min_row=2, max_row=3)
data = Reference(sheet, min_col=2, min_row=2, max_row=3)

# Для отображения данных на диаграмме
series = Series(data, title='Series 1')
pie_main.append(series)

s1 = pie_main.series[0]
s1.dLbls = DataLabelList()
s1.dLbls.showVal = True

pie_main.add_data(data, titles_from_data=True)
pie_main.set_categories(labels)
pie_main.title = 'Распределение обучившихся'
sheet.add_chart(pie_main, 'F1')


In [18]:
 # # Добавляем таблицу с по направлениям

sheet['A7'] = 'Вид обучения'
sheet['B7'] = 'Название программы'
sheet['C7'] = 'Количество обучившихся'

for row in df_counting_type_and_name_trainning.values.tolist():
    sheet.append(row)
# Получаем последние активные ячейки чтобы записывалось по порядку и не налазило друг на друга
min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

sheet[f'A{max_row + 2}'] = 'Общее распределение прошедших обучение по полу'
total_sex = counting_total_sex(dpo_df, po_df)
# Добавляем в файл таблицу с распределением по полам
for row in total_sex.values.tolist():
    sheet.append(row)

# Получаем последние активные ячейки чтобы записывалось по порядку и не налазило друг на друга
min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

# Добавляем таблицу с разбиением по возрастам 1-ПК
sheet[f'A{max_row + 2}'] = 'Распределение обучившихся по возрастным категориям 1-ПК'
age_distribution_dpo = counting_age_distribution_dpo(dpo_df)
for row in age_distribution_dpo.values.tolist():
    sheet.append(row)

# Добавляем круговую диаграмму
pie_age_dpo = PieChart()
# Для того чтобы не зависело от количества строк в предыдущих таблицах
labels = Reference(sheet, min_col=1, min_row=max_row + 3, max_row=max_row + 2 + len(age_distribution_dpo))
data = Reference(sheet, min_col=2, min_row=max_row + 3, max_row=max_row + 2 + len(age_distribution_dpo))
# Для отображения данных на диаграмме
series = Series(data, title='Series 1')
pie_age_dpo.append(series)

s1 = pie_age_dpo.series[0]
s1.dLbls = DataLabelList()
s1.dLbls.showVal = True

pie_age_dpo.add_data(data, titles_from_data=True)
pie_age_dpo.set_categories(labels)
pie_age_dpo.title = 'Распределение обучившихся по возрастным категориям 1-ПК'

sheet.add_chart(pie_age_dpo, f'F{max_row + 2}')

min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

# Добавляем таблицу с разбиением по возрастам 1-ПО
sheet[f'A{max_row + 4}'] = 'Распределение обучившихся по возрастным категориям 1-ПО'
age_distribution_po = counting_age_distribution_po(po_df)
for row in age_distribution_po.values.tolist():
    sheet.append(row)

# Добавляем круговую диаграмму
pie_age_po = PieChart()
# Для того чтобы не зависело от количества строк в предыдущих таблицах
labels = Reference(sheet, min_col=1, min_row=max_row + 5, max_row=max_row + 4 + len(age_distribution_po))
data = Reference(sheet, min_col=2, min_row=max_row + 5, max_row=max_row + 4 + len(age_distribution_po))
# Для отображения данных на диаграмме
series = Series(data, title='Series 1')
pie_age_po.append(series)

s1 = pie_age_po.series[0]
s1.dLbls = DataLabelList()
s1.dLbls.showVal = True

pie_age_po.add_data(data, titles_from_data=True)
pie_age_po.set_categories(labels)
pie_age_po.title = 'Распределение обучившихся по возрастным категориям 1-ПО'

sheet.add_chart(pie_age_po, f'F{max_row + 5}')

min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

sheet.column_dimensions['A'].width = 50
sheet.column_dimensions['B'].width = 30

In [19]:
# Сохраняем файл
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
wb.save(f'{path_to_end_folder_report}/Сводный отчет {current_time}.xlsx')